In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession\
    .builder\
    .appName("twitter_silver")\
    .getOrCreate()


23/07/15 11:28:06 WARN Utils: Your hostname, lucas-Estudos resolves to a loopback address: 127.0.1.1; using 192.168.0.102 instead (on interface wlp8s0)
23/07/15 11:28:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/07/15 11:28:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df_tweet = spark.read.json("/home/lucas/Área de Trabalho/Alura/EngenhariaDados/FormacaoAirflow/Datalake/Silver/twitter_datascience/tweet")

In [4]:
df_tweet.printSchema()

root
 |-- author_id: string (nullable = true)
 |-- conversation_id: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- like_count: long (nullable = true)
 |-- quote_count: long (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- text: string (nullable = true)
 |-- process_date: date (nullable = true)



In [5]:
from pyspark.sql  import  functions as f 

In [6]:
tweet_conversas = df_tweet.alias("tweet")\
        .groupBy(f.to_date("created_at").alias("created_date"))\
        .agg(
            f.countDistinct("author_id").alias("n_tweets"),
            f.sum("like_count").alias("n_like"),
            f.sum("quote_count").alias("n_quote"),
            f.sum("reply_count").alias("n_reply"),
            f.sum("retweet_count").alias("n_retweet")
        ).withColumn("weekday", f.date_format("created_date", "E"))


In [7]:
tweet_conversas.orderBy("created_date").show()

+------------+--------+------+-------+-------+---------+-------+
|created_date|n_tweets|n_like|n_quote|n_reply|n_retweet|weekday|
+------------+--------+------+-------+-------+---------+-------+
|  2023-07-08|      10|   553|    360|    509|      404|    Sat|
|  2023-07-09|      10|   497|    449|    418|      438|    Sun|
|  2023-07-10|       9|   404|    436|    434|      608|    Mon|
|  2023-07-11|      10|   537|    503|    499|      411|    Tue|
|  2023-07-12|      10|   428|    490|    327|      594|    Wed|
|  2023-07-13|       9|   549|    534|    546|      423|    Thu|
|  2023-07-14|      17|  1260|    991|    814|      650|    Fri|
+------------+--------+------+-------+-------+---------+-------+

